## Try out notebook for using julia code

test below is not faster than python code

In [69]:
using Glob, CSV, DataFrames, ProgressMeter, FileIO, MeshIO, PlyIO

# Get list of tiles
datafolder_raw_selection_clipped = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/raw_selection_clipped_10-25/"
tilenames = [joinpath(datafolder_raw_selection_clipped, f) for f in readdir(datafolder_raw_selection_clipped) if isfile(joinpath(datafolder_raw_selection_clipped, f)) && endswith(f, ".ply")]

# Get bounding boxes of segmented trees
bbox_filename = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/segmented/bounding_boxes.csv"
bbox = CSV.File(bbox_filename) |> DataFrame
bbox[:, :tree_nr] = 0:nrow(bbox)-1

# Make mapping of tree file to unique number
datafolder_segm = "/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/wythamwoods/segmented/"
filenames = [f for f in readdir(datafolder_segm) if isfile(joinpath(datafolder_segm, f)) && endswith(f, ".ply")]
tree_file2number = Dict(filename => i for (i, filename) in enumerate(filenames))
tree_number2file = Dict(i => filename for (i, filename) in enumerate(filenames))

# Iterate over all tiles
for i in 1:length(tilenames)
    # Read tile
    tilename = tilenames[i]
    tile = load_ply(tilename)
    x = tile["vertex"]["x"]
    y = tile["vertex"]["y"]
    z = tile["vertex"]["z"]

    # Get bounds of tile
    t_x_max = maximum(x)
    t_x_min = minimum(x)
    t_y_max = maximum(y)
    t_y_min = minimum(y)

    tile = hcat(x, y, z)

    # Get trees that fall (partly) within bounds of the tile
    bbox_in = bbox[(bbox[:, :x_min] .< t_x_max) .& (bbox[:, :x_max] .> t_x_min) .& (bbox[:, :y_min] .< t_y_max) .& (bbox[:, :y_max] .> t_y_min), :]
    print(bbox_in)
    trees_names_in = [tree_number2file[i] for i in bbox_in[!, :tree_nr]]

    # Read in point clouds of included trees
    trees_in = [load_ply(joinpath(datafolder_segm, tree_name)) for tree_name in trees_names_in]

    # Pre-allocate label array with value '-1'
    label = fill(-1, (size(tile, 1), 2))

    # Iterate over all trees within tile
    for (tree_name, tree_in) in zip(trees_names_in, trees_in)
        # Boolean list indicating where tile points occur as tree points
        tree = hcat(tree_in["vertex"]["x"], tree_in["vertex"]["y"], tree_in["vertex"]["z"])
        row_match = [row in eachrow(tree) for row in eachrow(tile)]
        # row_match = all(tile .== reshape(tree, (1, size(tree, 1), 3)), dims=2)[:, :, 1]

        # Allocate class and instance label
        label[row_match, 1] .= 1
        label[row_match, 2] .= tree_file2number[tree_name]
    end

    # Save label as csv file
    columns = ["label_class", "label_instance"]
    label_df = DataFrame(label, columns)
    print(label_df)
    # CSV.write(joinpath(dirname(tilename), "$(basename(splitext(tilename)[1]))_label.csv"), label_df)
end

In [68]:
arr1 = [1 2 3; 4 5 6; 7 8 9]
arr2 = [4 5 6; 10 11 12; 7 8 9]

# all(arr1 .== reshape(arr2, (1, size(arr2, 1), 3)), dims=2)[:, :, 1]

# [in(arr1[i, :], arr2) for i in 1:size(arr1, 1)]
[row in eachrow(arr2) for row in eachrow(arr1)]


3-element Vector{Bool}:
 0
 1
 1